In [101]:
#imports
import re
import pandas as pd
import numpy as np

In [102]:
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, log_loss

In [103]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [104]:
test.tail(5)

,index,text
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."
5681,5682,Vakatjhela iwebhusayidi yethu ku-www.


In [105]:
#CLEAN TEST
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [106]:
import preprocessor as p

#function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
    tempArr = []
    for line in df:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [107]:
test['text'] = test['text'].str.replace("[^a-zA-Z]", " ") 

C:\Users\SIPHOL~1\AppData\Local\Temp/ipykernel_5364/478941449.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test['text'] = test['text'].str.replace("[^a-zA-Z]", " ")


In [108]:
test.tail()

,index,text
5677,5678,You mark your ballot in private
5678,5679,Ge o ka kgetha ka bowena go se omi e Mofani k...
5679,5680,E Ka kopo etsa kgetho ya hao ka hloko hobane ...
5680,5681,TB ke bokudi ba PMB mme Morero o tla lefella ...
5681,5682,Vakatjhela iwebhusayidi yethu ku www


In [109]:
# clean training data
test_tweet = clean_tweets(test["text"])


In [110]:
test['text'] = test_tweet

In [111]:
test.tail()

,index,text
5677,5678,you mark your ballot in private
5678,5679,ge o ka kgetha ka bowena go se omi e mofani ka...
5679,5680,e ka kopo etsa kgetho ya hao ka hloko hobane h...
5680,5681,tb ke bokudi ba pmb mme morero o tla lefella t...
5681,5682,vakatjhela iwebhusayidi yethu ku www


In [112]:
train.head(3)

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...


In [113]:
''''def nguni_else(s):
    nguni = ['xho', 'zul', 'ssw', 'nbl']
    sotho = ['sot', 'tsn', 'nso']
    venda = ['ven', 'tso']
    germanic = ['eng' 'afr']
    
    for word in s.split():
        if word in nguni:
            return 'nguni'
        if word in sotho:
            return 'sotho'
        if word in venda:
            return 'venda'
    return 'germanic'
        ''' 

"'def nguni_else(s):\n    nguni = ['xho', 'zul', 'ssw', 'nbl']\n    sotho = ['sot', 'tsn', 'nso']\n    venda = ['ven', 'tso']\n    germanic = ['eng' 'afr']\n    \n    for word in s.split():\n        if word in nguni:\n            return 'nguni'\n        if word in sotho:\n            return 'sotho'\n        if word in venda:\n            return 'venda'\n    return 'germanic'\n        "

In [114]:
#train['root'] = train['lang_id'].apply(nguni_else)

In [115]:
train.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [116]:
#get dummies

#train_dummies = pd.get_dummies(train['root'])  

In [117]:
#combine my dfs

#train1 = pd.concat([train,train_dummies],axis=1)

In [118]:
train1 = train.copy()

In [119]:
#Adding a column for words that start with ka

def extract_ka(s):
    return len([part for part in s.split() if part.startswith('ka')])

In [120]:
train1['ka_count'] = train1['text'].apply(extract_ka)
train1.head(1)

,lang_id,text,ka_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0


In [121]:
#DO FOR TEST
test['ka_count'] = test['text'].apply(extract_ka)
test.head(1)

,index,text,ka_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0


In [122]:
#Adding a column for words that start with ngo

def extract_ngo(s):
    return len([part for part in s.split() if part.startswith('ngo')])

In [123]:
train1['ngo_count'] = train1['text'].apply(extract_ngo)
train1.head(1)

,lang_id,text,ka_count,ngo_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1


In [124]:
#DO FOR TEST
test['ngo_count'] = test['text'].apply(extract_ngo)
test.head(1)

,index,text,ka_count,ngo_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0


In [125]:
#Adding a column for words that start with ngo

def extract_zw(s):
    return len([part for part in s.split() if part.startswith('zw')])

In [126]:
train1['zw_count'] = train1['text'].apply(extract_zw)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0


In [127]:
#DO FOR TEST
test['zw_count'] = test['text'].apply(extract_zw)
test.head(1)

,index,text,ka_count,ngo_count,zw_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0


In [128]:
#Adding a column for words that start with mm

def extract_mm(s):
    return len([part for part in s.split() if part.startswith('mm')])

In [129]:
train1['mm_count'] = train1['text'].apply(extract_mm)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0


In [130]:
#DO FOR TEST
test['mm_count'] = test['text'].apply(extract_mm)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1


In [131]:
#Adding a column for words that start with ge

def extract_ge(s):
    return len([part for part in s.split() if part.startswith('ge')])

In [132]:
train1['ge_count'] = train1['text'].apply(extract_ge)

In [133]:
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0


In [134]:
#FOR TEST
test['ge_count'] = test['text'].apply(extract_ge)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0


In [135]:
#train1[train1['ge_count'] > 0].head()

In [136]:
#Adding a column for words that start with go

def extract_go(s):
    return len([part for part in s.split() if part.startswith('go')])

In [137]:
train1['go_count'] = train1['text'].apply(extract_go)

In [138]:
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0


In [139]:
#DO FOR TEST

test['go_count'] = test['text'].apply(extract_go)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0


In [140]:
#Adding a column for words that start with nie

def extract_nie(s):
    return len([part for part in s.split() if part.startswith('nie')])

In [141]:
train1['count_nie'] = train1['text'].apply(extract_nie)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0


In [142]:
#DO FOR TEST
test['count_nie'] = test['text'].apply(extract_nie)
test.head()

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...,0,1,0,0,0,0,0
2,3,tshivhumbeo tshi fana na ngano dza vhathu,0,0,0,0,0,0,0
3,4,kube inja nelikati betingevakala kutsi titsini...,0,0,0,0,0,0,0
4,5,winste op buitelandse valuta,0,0,0,0,0,0,0


In [143]:
#Adding a column for words that start with o

def extract_o(s):
    return len([part for part in s.split() if part.startswith('o')])

In [144]:
train1['o_count'] = train1['text'].apply(extract_o)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1


In [145]:
test['o_count'] = test['text'].apply(extract_o)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0


In [146]:
#Adding a column for words length of text

def extract_len(s):
    return len([part for part in s.split()])

In [147]:
train1['text_len'] = train1['text'].apply(extract_len)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24


In [148]:
#DO FOR TEST
test['text_len'] = test['text'].apply(extract_len)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10


In [149]:
#train1.tail()

In [150]:
#Adding a column for words that start with ge

def extract_e(s):
    #word = "e"
    return len([part for part in s.split() if part.startswith('e')])

In [151]:
train1['e_count'] = train1['text'].apply(extract_e)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1


In [152]:
#DO FOR TEST

test['e_count'] = test['text'].apply(extract_e)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1


In [153]:
#train1.head()

In [154]:
#Adding a column for words that start with van

def extract_van(s):
    return len([part for part in s.split() if part.startswith('van')])

In [155]:
train1['van_count'] = train1['text'].apply(extract_van)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0


In [156]:
#DO FOR TEST
test['van_count'] = test['text'].apply(extract_van)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0


In [157]:
#Adding a column for words that start with g

def extract_g(s):
    return len([part for part in s.split() if part.startswith('g')])

In [158]:
train1['g_word_count'] = train1['text'].apply(extract_g)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0


In [159]:
test['g_word_count'] = test['text'].apply(extract_g)
test.head()

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...,0,1,0,0,0,0,0,0,10,2,0,0
2,3,tshivhumbeo tshi fana na ngano dza vhathu,0,0,0,0,0,0,0,0,7,0,0,0
3,4,kube inja nelikati betingevakala kutsi titsini...,0,0,0,0,0,0,0,0,7,0,0,0
4,5,winste op buitelandse valuta,0,0,0,0,0,0,0,1,4,0,0,0


In [160]:
#train1.head()

In [161]:
#Adding a column for words that start with u

def extract_u(s):
    return len([part for part in s.split() if part.startswith('u')])

In [162]:
train1['u_word_count'] = train1['text'].apply(extract_u)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2


In [163]:
#DO FOR TEST
test['u_word_count'] = test['text'].apply(extract_u)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0


In [164]:
#Adding a column for words that start with kg

def extract_kg(s):
    return len([part for part in s.split() if part.startswith('kg')])

In [165]:
train1['kg_word_count'] = train1['text'].apply(extract_kg)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2,0


In [166]:
#DO FOR TEST
test['kg_word_count'] = test['text'].apply(extract_kg)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0,2


In [167]:
#Adding a column for words that start with aa

def extract_a(s):
    return len([part for part in s.split() if part.startswith('a')])

In [168]:
train1['a_word_count'] = train1['text'].apply(extract_a)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2,0,3


In [169]:
#DO FOR TEST
test['a_word_count'] = test['text'].apply(extract_a)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0,2,3


In [170]:
#Adding a column for words that start with i

def extract_i(s):
    return len([part for part in s.split() if part.startswith('i')])

In [171]:
train1['i_word_count'] = train1['text'].apply(extract_i)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2,0,3,3


In [172]:
#DO FOR TEST
test['i_word_count'] = test['text'].apply(extract_i)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0,2,3,0


In [173]:
#Adding a column for words that start with i

def extract_ya(s):
    return len([part for part in s.split() if part.startswith('ya')])

In [174]:
train1['ya_word_count'] = train1['text'].apply(extract_ya)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2,0,3,3,1


In [175]:
#DO FOR TEST
test['ya_word_count'] = test['text'].apply(extract_ya)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0,2,3,0,0


In [176]:
#train1[train1['lang_id']=='ven']

In [177]:
#Add target

def target(s):
    for word in s.split():
        if word == "afr":
            return 1
        if word == "eng":
            return 2
        if word == "zul":
            return 3
        if word == "xho":
            return 4
        if word == "ssw":
            return 5
        if word == "nbl":
            return 6
        if word == "sot":
            return 7
        if word == "tsn":
            return 8
        if word == "nso":
            return 9
        if word == "ven":
            return 10
        if word == "tso":
            return 11
        

In [64]:
train1['target'] = train1['lang_id'].apply(target)

In [178]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   lang_id        33000 non-null  object
 1   text           33000 non-null  object
 2   ka_count       33000 non-null  int64 
 3   ngo_count      33000 non-null  int64 
 4   zw_count       33000 non-null  int64 
 5   mm_count       33000 non-null  int64 
 6   ge_count       33000 non-null  int64 
 7   go_count       33000 non-null  int64 
 8   count_nie      33000 non-null  int64 
 9   o_count        33000 non-null  int64 
 10  text_len       33000 non-null  int64 
 11  e_count        33000 non-null  int64 
 12  van_count      33000 non-null  int64 
 13  g_word_count   33000 non-null  int64 
 14  u_word_count   33000 non-null  int64 
 15  kg_word_count  33000 non-null  int64 
 16  a_word_count   33000 non-null  int64 
 17  i_word_count   33000 non-null  int64 
 18  ya_word_count  33000 non-n

In [179]:
#Adding a column for words that start with tshi

def extract_tshi(s):
    return len([part for part in s.split() if part.startswith('tshi')])

In [180]:
train1['tshi_word_count'] = train1['text'].apply(extract_tshi)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,24,1,0,0,2,0,3,3,1,0


In [181]:
#DO FOR TEST
test['tshi_word_count'] = test['text'].apply(extract_tshi)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,text_len,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,10,1,0,0,0,2,3,0,0,0


In [182]:
#Adding a column for words that start with tshi

def extract_dz(s):
    return len([part for part in s.split() if part.startswith('dz')])

In [183]:
train1['dz_word_count'] = train1['text'].apply(extract_dz)
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,...,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count,dz_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,...,1,0,0,2,0,3,3,1,0,0


In [184]:
#DO FOR TEST
test['dz_word_count'] = test['text'].apply(extract_dz)
test.head(1)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,...,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count,dz_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,...,1,0,0,0,2,3,0,0,0,0


In [185]:
test.head(10)

,index,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,...,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count,dz_word_count
0,1,mmasepala fa maemo a a kgethegileng a letlelel...,0,0,0,1,0,0,0,0,...,1,0,0,0,2,3,0,0,0,0
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...,0,1,0,0,0,0,0,0,...,2,0,0,3,0,0,1,0,0,0
2,3,tshivhumbeo tshi fana na ngano dza vhathu,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,1
3,4,kube inja nelikati betingevakala kutsi titsini...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,winste op buitelandse valuta,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,6,ke feela dilense t e hlakilego t a pono e tee ...,1,0,0,0,1,5,0,2,...,5,0,7,0,2,5,0,2,0,0
6,7,fn am final gems birthing options zulu txt fn,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
7,8,ntjhafatso ya konteraka ya mosebetsi etsa bonn...,0,1,0,0,0,0,0,0,...,2,0,0,0,0,0,0,2,0,0
8,9,u gems uhlinzeka ngezinzuzo zemithi yezifo ezi...,0,0,0,0,2,0,0,1,...,5,0,2,6,0,1,3,0,0,0
9,10,so on occasion are statistics misused,0,0,0,0,0,0,0,2,...,0,0,0,0,0,1,0,0,0,0


### MODELLING

In [186]:
#MODEL

In [187]:
train1.head(1)

,lang_id,text,ka_count,ngo_count,zw_count,mm_count,ge_count,go_count,count_nie,o_count,...,e_count,van_count,g_word_count,u_word_count,kg_word_count,a_word_count,i_word_count,ya_word_count,tshi_word_count,dz_word_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,0,1,0,0,0,0,0,1,...,1,0,0,2,0,3,3,1,0,0


In [189]:
X = train1.drop(['lang_id'],axis=1)
y = train1.lang_id.values

In [190]:
len(X), len(y)

(33000, 33000)

In [199]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english',
                             min_df=1,
                             max_df=0.5,
                             ngram_range=(1, 1))

train_x_vecs = vectorizer.fit_transform(X["text"])

In [200]:
train_x_vecs.shape

(33000, 141680)

In [201]:
vectorizer2 = CountVectorizer(vocabulary = vectorizer.get_feature_names(), stop_words='english',
                             min_df=1,
                             max_df=0.5,
                             ngram_range=(1, 1))

In [202]:
test_x_vecs = vectorizer2.transform(test.text)

In [203]:
test_x_vecs.shape

(5682, 141680)

In [204]:
#Split the data

X_train, X_test, y_train, y_test = train_test_split(train_x_vecs, y, test_size=0.2, random_state = 42)

### Model 1: Logistic Regression

In [205]:
lm_full = LogisticRegression()

In [206]:
lm_full.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [207]:
lm_pred = lm_full.predict(X_test)

In [208]:
lm_pred

array(['sot', 'nso', 'eng', ..., 'afr', 'nso', 'eng'], dtype=object)

In [209]:
print(classification_report(y_test, lm_pred))

              precision    recall  f1-score   support

         afr       1.00      0.99      1.00       583
         eng       0.99      1.00      1.00       615
         nbl       0.98      0.99      0.98       583
         nso       1.00      0.99      0.99       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       0.99      0.99      0.99       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       0.99      0.99      0.99       609
         zul       0.98      0.98      0.98       590

    accuracy                           0.99      6600
   macro avg       0.99      0.99      0.99      6600
weighted avg       0.99      0.99      0.99      6600



In [210]:
clf_log = LogisticRegression()
clf_log.fit(train_x_vecs, y)
fin_pred_log = clf_log.predict(test_x_vecs)

In [213]:
import pickle
#pickle.dump(clf_log, open('clf'))

In [214]:
len(fin_pred_log)

5682

In [218]:
LogRegrF = pd.DataFrame(list(zip(test.index + 1, fin_pred_log)), columns = ['index','lang_id'])
LogRegrF.head()

,index,lang_id
0,1,eng
1,2,nbl
2,3,xho
3,4,ssw
4,5,eng


In [231]:
#train1[train1['lang_id'] == 'nbl']

In [219]:
LogRegrF.head(10)

,index,lang_id
0,1,eng
1,2,nbl
2,3,xho
3,4,ssw
4,5,eng
5,6,nso
6,7,eng
7,8,sot
8,9,zul
9,10,eng


In [220]:
LogRegrF.to_csv('first sub submission.csv', index_label = False, index = False)

### Model 2: Naive Bayes Miltinomial (my final choice)

In [221]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)

print("The accuracy score of the MNB is:", accuracy_score(y_test, y_pred))
print("The f1 score is:", f1_score(y_test, y_pred, average = 'micro'))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))


The accuracy score of the MNB is: 0.9989393939393939
The f1 score is: 0.9989393939393939


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      1.00      1.00       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       1.00      1.00      1.00       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [222]:
mnb = MultinomialNB()
mnb.fit(train_x_vecs, y)
y_pred_mnb = mnb.predict(test_x_vecs)

In [223]:
y_pred_mnb

array(['tsn', 'nbl', 'ven', ..., 'sot', 'sot', 'nbl'], dtype='<U3')

In [224]:
mnb1 = pd.DataFrame(list(zip(test.index + 1, y_pred_mnb)), columns = ['index','lang_id'])
mnb1.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [225]:
mnb1.to_csv('second sub mnb submission.csv', index_label = False, index = False)

### Model 3: Random Forest Classifier

In [243]:


ram = RandomForestClassifier()
ram.fit(X_train, y_train)
y_pred = ram.predict(X_test)

print("The accuracy score of the SVC is:", accuracy_score(y_test, y_pred))
print("The f1 score is:", f1_score(y_test, y_pred, average = 'micro'))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

The accuracy score of the SVC is: 0.9828787878787879
The f1 score is: 0.9828787878787879


Classification Report:

               precision    recall  f1-score   support

         afr       0.99      1.00      1.00       583
         eng       0.95      1.00      0.97       615
         nbl       0.98      0.96      0.97       583
         nso       1.00      0.99      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       0.99      0.96      0.98       584
         tsn       0.99      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       0.98      0.96      0.97       609
         zul       0.94      0.95      0.95       590

    accuracy                           0.98      6600
   macro avg       0.98      0.98      0.98      6600
weighted avg       0.98      0.98      0.98      6600



### Model 4: Support Vector Classifier

In [227]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', C=1)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print("The accuracy score of the SVC is:", accuracy_score(y_test, y_pred))
print("The f1 score is:", f1_score(y_test, y_pred, average = 'micro'))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

The accuracy score of the SVC is: 0.9939393939393939
The f1 score is: 0.9939393939393939


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      0.99      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.98      0.99      0.99       583
         nso       1.00      0.99      0.99       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       0.99      0.99      0.99       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       0.99      0.99      0.99       609
         zul       0.98      0.98      0.98       590

    accuracy                           0.99      6600
   macro avg       0.99      0.99      0.99      6600
weighted avg       0.99      0.99      0.99      6600



In [236]:
svc = SVC(kernel='linear', C=1)
svc.fit(train_x_vecs, y)
y_pred_SVM = svc.predict(test_x_vecs)

In [237]:
y_pred_SVM

array(['eng', 'nbl', 'xho', ..., 'sot', 'sot', 'ssw'], dtype=object)

In [238]:
svc1 = pd.DataFrame(list(zip(test.index + 1, y_pred_SVM)), columns = ['index','lang_id'])
svc1.head()

,index,lang_id
0,1,eng
1,2,nbl
2,3,xho
3,4,ssw
4,5,eng


In [240]:
svc1.to_csv('fourth sub svc submission.csv', index_label = False, index = False)

### Model 5: Naive Bayes Bernoulli (second best performer)

In [228]:
gnb = BernoulliNB()
gnb.fit(X_train, y_train)
y_pred_B = gnb.predict(X_test)

print("The accuracy score of the GNB is:", accuracy_score(y_test, y_pred_B))
print("The f1 score is:", f1_score(y_test, y_pred_B, average = 'micro'))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred_B))

The accuracy score of the GNB is: 0.9989393939393939
The f1 score is: 0.9989393939393939


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      1.00      1.00       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       1.00      1.00      1.00       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [232]:
gnb = BernoulliNB()
gnb.fit(train_x_vecs, y)
y_pred_B = gnb.predict(test_x_vecs)

In [233]:
y_pred_B

array(['tsn', 'nbl', 'ven', ..., 'sot', 'sot', 'nbl'], dtype='<U3')

In [234]:
bnb = pd.DataFrame(list(zip(test.index + 1, y_pred_B)), columns = ['index','lang_id'])
bnb.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,eng


In [235]:
bnb.to_csv('third sub ben submission.csv', index_label = False, index = False)

### Model 6: Enhanced Log Reg

In [241]:

clf_log = LogisticRegression(C=5)
clf_log.fit(X_train, y_train)
y_pred = clf_log.predict(X_test)

print("The accuracy score of the Log_Reg is:", accuracy_score(y_test, y_pred))
print("The f1 score is:", f1_score(y_test, y_pred, average = 'micro'))
print("\n\nClassification Report:\n\n", classification_report(y_test, y_pred))

C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


The accuracy score of the Log_Reg is: 0.9940909090909091
The f1 score is: 0.9940909090909091


Classification Report:

               precision    recall  f1-score   support

         afr       1.00      0.99      1.00       583
         eng       0.99      1.00      1.00       615
         nbl       0.98      0.98      0.98       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      0.99      0.99       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       0.99      0.99      0.99       609
         zul       0.98      0.98      0.98       590

    accuracy                           0.99      6600
   macro avg       0.99      0.99      0.99      6600
weighted avg       0.99      0.99      0.99      6600

